<a href="https://colab.research.google.com/github/ngockhanh5110/nlp-vietnamese-text-summarization/blob/main/notebooks/training_vietnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [170]:
# Authenticate
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [171]:
OUTPUT_DIR = '/content/training/'

In [3]:
import random

random.seed(0)

In [ ]:
!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

In [4]:
%%capture

# !wget 'https://github.com/CLC-HCMUS/ViMs-Dataset/raw/master/ViMs.zip'
# !unzip 'ViMs.zip'

# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [5]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

## Processing data

In [6]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [122]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [144]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
            'original': original, 
            'summary': summary}

In [156]:
read_content(train_paths[0])

{'file': '/content/vietnews-master/data/train_tokenized/033889.txt.seg',
 'original': 'Theo RT , liên_quân do Mỹ dẫn_đầu được cho là đã không_kích các vị_trí của quân_đội Syria tại khu_vực Al_Bukamal thuộc tỉnh Deir ez - Zor , phía Đông_Syria . Thông_tin trên được hãng thông_tấn nhà_nước Syria SANA trích_dẫn nguồn tin quân_sự cho_hay . Theo đó , cuộc không_kích nhằm vào một_số vị_trí của quân_đội Syria tại al - Harra , phía Đông nam khu_vực Al_Bukamal . Theo nguồn_tin này , vụ không_kích dẫn đến tổn_thất về nhân_lực của quân_đội Syria , tuy_nhiên con_số thương_vong cụ_thể chưa được công_bố . Vụ không_kích này có_thể do một_số máy_bay_không_người_lái ( UAV ) của Mỹ thực_hiện , đồng_thời vụ không_kích này được cho là nhằm vào một_số vị_trí của lực_lượng Iraq và quân_đội Chính_phủ Syria tại khu_vực giữa Al_Bukamal và Tanf . Tuy_nhiên , hiện Lầu_Năm_Góc phủ_nhận có liên_quan tới vụ_việc . Đây không phải là vụ không kích đầu_tiên nhằm vào lực_lượng quân_đội Syria và các lực_lượng ủng_hộ Chí

In [164]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [165]:
train_df = get_dataframe(train_paths)

In [166]:
val_df = get_dataframe(val_paths)
test_df = get_dataframe(test_paths)

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [14]:
%%capture
!pip install datasets==1.0.2
!pip install transformers

import datasets
import transformers

In [167]:
from transformers import RobertaTokenizerFast,AutoTokenizer

# phobert = AutoModel.from_pretrained("vinai/phobert-base")

# For transformers v4.x+: 
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)


In [168]:
from sklearn.model_selection import train_test_split

train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [183]:
batch_size=16  # change to 16 for full training
encoder_max_length=256
decoder_max_length=64

def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [                                                                                 
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

### **Warm-starting the Encoder-Decoder Model**

In [184]:
from transformers import EncoderDecoderModel

# set encoder decoder tying to True
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("vinai/phobert-base", "vinai/phobert-base", tie_encoder_decoder=True)

In [185]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 64
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size  

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [174]:
%%capture
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score

from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

In [175]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [176]:
import datasets

In [177]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Cool! Finally, we start training.

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=200,  # set to 2000 for full training
    save_steps=5000,  # set to 500 for full training
    eval_steps=7500,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=50,
    fp16=True, 
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
)
trainer.train()

The `config.pad_token_id` is `None`. Using `config.eos_token_id` = 2 for padding..
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss
200,9.417800


In [ ]:
!gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [116]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

In [117]:
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1","rouge2","rougeL"])

In [118]:
for key,value in rouge_output.items():
  print(key)
  print(value.mid)

rouge1
Score(precision=0.6156883807011324, recall=0.6143045549259318, fmeasure=0.5906751582344583)
rouge2
Score(precision=0.3081612284527864, recall=0.3055766169704671, fmeasure=0.29421411955015214)
rougeL
Score(precision=0.4081913955092156, recall=0.405197179436527, fmeasure=0.3907000488647073)


In [119]:
i = 154
print('Prediction: ',pred_str[i])
print('Truth: ',label_str[i])
print('Content: ',test_data[i]['original'])

Prediction:  Tên trộm đã đột_nhập đền Xã_Tắc ( phường Ka_Long, TP. Móng_Cái ) lấy trộm khoảng 300 triệu đồng từ hòm công_đức.
Truth:  Camera tại đền Xã_Tắc ( Quảng_Ninh ) ghi lại hình_ảnh hai thanh_niên cạy cửa đền , dùng thanh sắt và đục phá hỏng 3 chiếc hòm công_đức lấy đi số tiền ước_tính 300 triệu đồng .
Content:  Công_an thành_phố Móng_Cái ( Quảng_Ninh ) đang truy_tìm hai nghi phạm trộm tài_sản tại đền Xã_Tắc ở phường Ka_Long . Trước đó , khoảng 2h15 ngày 28/5 , hai thanh_niên đột_nhập đền , cạy cửa , phá hỏng 3 chiếc hòm công_đức lấy đi toàn_bộ tiền bên trong . Tài_sản bị mất ước_chừng 300 triệu đồng , chủ_yếu có mệnh_giá 10.000-50.000 đồng . Hình_ảnh do camera an_ninh ghi lại cho thấy hai nghi phạm tuổi từ 25 đến 30 . Phân_tích các dữ_liệu , cảnh_sát nhận_định , một người cao khoảng 1m65 , miệng rộng , má hóp , dáng gầy , mặc áo_phông ngắn tay màu nâu , đội mũ lưỡi chai màu ghi . Người thứ hai cao khoảng 1m70 , dáng gầy , mặc quần vải màu đen , áo sơmi dài tay tối màu , đội mũ c

In [120]:
test_data[i]['file']

'/content/ViMs/original/Cluster_256/original/1671.txt'

In [79]:
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [112]:
text = '''
 Ngày 4/2, lãnh đạo thị xã Hoàng Mai (Nghệ An) cho biết, cơ quan chức năng thị xã vừa phối hợp với đơn vị chức năng tỉnh cách ly, lấy mẫu xét nghiệm cho 1 nam sinh viên trường Đại học FPT có biểu hiện ho sốt sau khi đi từ Hà Nội về nhà.

Trước đó vào ngày 31/1 nam sinh C.T.Ph. (20 tuổi, trú TX. Hoàng Mai, Nghệ An; Sinh viên trường Đại học FPT cơ sở Mỹ Đình) cùng 1 người bạn đi xe máy từ Hà Nội về quê nhà.

Lúc 22h ngày 31/1 nam sinh này về đến quê nhà nhưng không khai báo y tế. Từ ngày 1-3/2, nam sinh Ph. có đi một số nơi trên địa bàn thị xã và tiếp xúc với khoảng 12 người.


Khoảng 20h ngày 3/2, nam sinh này có dấu hiệu sốt 39 độ, đau đầu, đau rát họng, đau mỏi cơ, khó thở nhẹ. Nam sinh này sau đó được đưa vào Bệnh viện Phong da liễu (TX. Hoàng Mai) để khám và cách ly y tế.

Nhận được tin báo, Trung tâm Kiểm soát bệnh tật tỉnh Nghệ An đã trực tiếp ra lấy mẫu bệnh phẩm để xét nghiệm Covid-19. Dự kiến trong chiều cùng ngày sẽ có kết quả.

Hiện cơ quan chức năng đang rà soát những người tiếp xúc gần với nam sinh Ph. để có phương án theo dõi, cách ly và lấy mẫu xét nghiệm.

Trước đó vào đêm 31/1, một chuyến xe khách chở 22 người từ Hà Nội về Nghệ An trong đó có nhiều sinh viên Đại học FPT đi về tại phường Lê Lợi, TP. Vinh (Nghệ An). Cơ quan chức năng sau đó đã đưa 1 trường hợp F1 trên xe khách này đi cách ly tại Trung tâm Y tế huyện Nam Đàn. Riêng những trường hợp còn lại được theo dõi tại nhà.

Sau khi cách ly, các trường hợp này đã được lấy mẫu xét nghiệm và cho kết quả âm tính với Covid-19.
 '''

In [113]:
text = rdrsegmenter.tokenize(text)
text = ' '.join([' '.join(x) for x in text])
text

'Ngày 4/2 , lãnh_đạo thị_xã Hoàng_Mai ( Nghệ_An ) cho biết , cơ_quan_chức_năng thị_xã vừa phối_hợp với đơn_vị chức_năng tỉnh cách_ly , lấy mẫu xét_nghiệm cho 1 nam sinh_viên trường Đại_học FPT có biểu_hiện ho sốt sau khi đi từ Hà_Nội về nhà . Trước đó vào ngày 31/1 nam_sinh C.T.Ph. ( 20 tuổi , trú TX . Hoàng_Mai , Nghệ_An ; Sinh_viên trường Đại_học FPT cơ_sở Mỹ_Đình ) cùng 1 người bạn đi xe_máy từ Hà_Nội về quê nhà . Lúc 22h ngày 31/1 nam_sinh này về đến quê nhà nhưng không khai_báo y_tế . Từ ngày 1-3/2 , nam_sinh Ph . có đi một_số nơi trên địa_bàn thị_xã và tiếp_xúc với khoảng 12 người . Khoảng 20h ngày 3/2 , nam_sinh này có dấu_hiệu sốt 39 độ , đau_đầu , đau rát họng , đau mỏi cơ , khó thở nhẹ . Nam_sinh này sau đó được đưa vào Bệnh_viện Phong da_liễu ( TX . Hoàng_Mai ) để khám và cách_ly y_tế . Nhận được tin báo , Trung_tâm Kiểm_soát bệnh_tật tỉnh Nghệ_An đã trực_tiếp ra lấy mẫu bệnh_phẩm để xét_nghiệm Covid-19 . Dự_kiến trong chiều cùng ngày sẽ có kết_quả . Hiện cơ_quan_chức_năng đ

In [114]:
inputs = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

# all special tokens including will be removed
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str

['Trong người bạn gái của một cán_bộ Công_an tỉnh Nghệ_An cho biết, đơn_vị đang khẩn_trương điều_tra, xác_minh nguyên_nhân khiến một nam thanh_niên tử_vong.']